In [2]:
import numpy as np 
import pandas as pd
import math

# t=experiation time, p= ATM strike_price 4154.52
# p_c=call price, p_p=put price
# r=Risk-free interest rate to expiration
def calculate_vix(t,p,p_c,p_p,r):
    T=t/365
    F=p+math.exp(r*T)*(p_c-p_p)

In [3]:
call=pd.read_csv("D://stock_data/call_option.csv")
call["Strike"]

,Strike,Moneyness,Bid,Midpoint,Ask,Last,Change,%Chg,Volume,Open Int,OI Chg,Volume Open Interest Ratio,IV,Type,Last Trade
0,"4,105.00",1.19%,103.9,104.40,104.9,108.93,-2.57,-2.30%,40,327,-4,14.76%,Call,15:09 ET,14.33%
1,"4,110.00",1.07%,100.3,100.80,101.3,105.32,1.32,1.27%,61,413,46,14.66%,Call,15:09 ET,14.33%
2,"4,115.00",0.95%,96.8,97.30,97.8,94.71,-9.29,-8.93%,3,717,29,14.56%,Call,11:51 ET,14.33%
3,"4,120.00",0.83%,93.3,93.80,94.3,86.70,-11.94,-12.10%,7,"1,159",105,14.46%,Call,09:50 ET,14.33%
4,"4,125.00",0.71%,89.8,90.30,90.8,89.84,-6.16,-6.42%,29,586,9,14.24%,Call,16:14 ET,14.33%
5,"4,130.00",0.59%,86.5,86.95,87.4,91.10,0.08,0.09%,13,907,90,14.25%,Call,14:17 ET,14.33%
6,"4,135.00",0.47%,83.1,83.60,84.1,83.62,-3.83,-4.38%,52,312,94,14.15%,Call,16:14 ET,14.33%
7,"4,140.00",0.35%,79.8,80.30,80.8,84.30,3.70,4.59%,63,433,25,14.04%,Call,15:29 ET,14.33%
8,"4,145.00",0.23%,76.6,77.10,77.6,77.35,-3.55,-4.39%,37,306,48,14.00%,Call,12:13 ET,14.33%
9,"4,150.00",0.11%,73.5,73.95,74.4,73.72,-5.83,-7.33%,392,"2,462",-63,13.80%,Call,16:14 ET,14.33%
